In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch import nn
from tqdm.notebook import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
import torch

In [ ]:
torch.cuda.device_count()

In [ ]:
m = torch.tensor([[1,2],[3,4]], dtype=torch.float32, device='cuda:0')

In [ ]:
torch.tensor([1,2]).device

In [ ]:
m.device

In [ ]:
m * 2

In [ ]:
a = torch.tensor([1,2.])

In [ ]:
m

In [ ]:
torch.sin(m)

In [ ]:
print(torch.relu(m))

In [ ]:
print(torch.sum(m,dim=1))

In [ ]:
print(torch.softmax(m,dim=1))

In [ ]:
x = torch.tensor([-1.,0,2],requires_grad=True)
c = x.cos()
s = c.sum()

In [ ]:
print(x)
print(c)
print(s)

In [ ]:
s.backward()

In [ ]:
x.grad

In [ ]:
print(-x.sin())

In [ ]:
housing_df = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')

In [ ]:
housing_df.head(5)

In [ ]:
housing_df.shape

In [ ]:
housing_df.isna().sum()

In [ ]:
housing_df.isna().any()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
housing_df.total_bedrooms.isna()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [ ]:
oh = OneHotEncoder(sparse=False)
tr = oh.fit_transform(housing_df['ocean_proximity'].values.reshape(-1,1))

In [ ]:
oh.categories_

In [ ]:
class HousingPipeline():
    def __init__(self):
        self.bedroom_coef = None
        self.oh = None
        self.target_mean = None
        self.target_std = None
    
    def transform_train(self, housing_df):
        new_df = housing_df.copy()
        bedrooms_regression = LinearRegression(fit_intercept=False)
        
        mask = ~(housing_df.total_bedrooms.isna().values)
        
        bedroom_input = housing_df.total_rooms.values[mask].reshape(-1,1)
        bedroom_target = housing_df.total_bedrooms.values[mask]
        bedrooms_regression.fit(bedroom_input, bedroom_target)
        self.bedroom_coef = bedrooms_regression.coef_.item()
        
        new_df['total_bedrooms'][~mask] = self.bedroom_coef * new_df['total_rooms'][~mask]
        
        new_df['avg_rooms'] = new_df['total_rooms'] / new_df['households']
        new_df['avg_bedrooms'] = new_df['total_bedrooms'] / new_df['households']
        new_df = new_df.drop(['total_bedrooms','total_rooms'],axis=1)
        
        self.oh = OneHotEncoder(sparse=False)
        oh_transformed = self.oh.fit_transform(new_df['ocean_proximity'].values.reshape(-1,1))
        for i, f in enumerate(self.oh.categories_):
            for j, category in enumerate(f):
                assert i == 0
                new_df[category] = oh_transformed[:,j]
        
        
        
        target = new_df['median_house_value'].values
        self.target_mean = np.mean(target).item()
        self.target_std = np.std(target).item()
        
        new_df = new_df.drop(['ocean_proximity','median_house_value'],axis=1)
        return new_df, target
    
    def transform_test(self, housing_df):
        new_df = housing_df.copy()
        nan_mask = (housing_df.total_bedrooms.isna().values)
        new_df['total_bedrooms'][nan_mask] = self.bedroom_coef * new_df['total_rooms'][nan_mask]
        new_df['avg_rooms'] = new_df['total_rooms'] / new_df['households']
        new_df['avg_bedrooms'] = new_df['total_bedrooms'] / new_df['households']
        new_df = new_df.drop(['total_bedrooms','total_rooms'],axis=1)
        
        oh_transformed = self.oh.transform(new_df['ocean_proximity'].values.reshape(-1,1))
        for i, f in enumerate(self.oh.categories_):
            for j, category in enumerate(f):
                assert i == 0
                new_df[category] = oh_transformed[:,j]
                
        target = new_df['median_house_value'].values
        new_df = new_df.drop(['ocean_proximity','median_house_value'],axis=1)
        return new_df, target
        
        
        

In [ ]:
housing_train, housing_test = train_test_split(housing_df,test_size=0.2)

In [ ]:
housing_pipeline = HousingPipeline()

In [ ]:
X_train, y_train = housing_pipeline.transform_train(housing_train)
X_test, y_test = housing_pipeline.transform_test(housing_test)

In [ ]:
lreg = LinearRegression().fit(X_train.values, y_train)

In [ ]:
lreg.score(X_test.values, y_test)

In [ ]:
xgb = XGBRegressor().fit(X_train.values, y_train)

In [ ]:
xgb.score(X_test.values, y_test)

In [ ]:
X_train_torch, y_train_torch = torch.tensor(X_train.values, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)

In [ ]:
X_test_torch, y_test_torch = torch.tensor(X_test.values, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
train_dataset = TensorDataset(X_train_torch, y_train_torch)
test_dataset = TensorDataset(X_test_torch, y_test_torch)

In [ ]:
X_train_torch.size()

In [ ]:
class HousingNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(input_size)
        self.linear1 = nn.Linear(input_size, hidden_size1)
        self.bn1 = nn.BatchNorm1d(hidden_size1)
        self.linear2 = nn.Linear(hidden_size1, hidden_size2)
        self.output_layer =  nn.Linear(hidden_size2, 1)
        
    def forward(self, X):
        a = self.linear1(self.bn0(X))
        h1 = torch.relu(self.bn1(a))
        h2 = torch.relu(self.linear2(h1))
        output = self.output_layer(h2)
        return output.view(-1)
        

In [ ]:
housing_net = HousingNet(13, 64, 64)

In [ ]:
criterion = nn.MSELoss()

In [ ]:
optimizer = torch.optim.Adam(housing_net.parameters(), lr=3e-4)

In [ ]:
train_batch_size = 128
test_batch_size = 512

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=test_batch_size)

In [ ]:
n_epoch = 50

In [ ]:
housing_pipeline.target_mean

In [ ]:
from sklearn import metrics

In [ ]:
def train_network(net, train_loader, test_loader, criterion, test_criterion, optimizer, n_epochs, device):
    for epoch in range(n_epochs):
        print(next(iter(net.parameters())).device)
        net.train()
        for i, (X_batch, y_batch) in enumerate(train_loader):
            optimizer.zero_grad()
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = net(X_batch)
            
            y_batch = (y_batch - housing_pipeline.target_mean) / housing_pipeline.target_std
            
            loss = criterion(y_pred, y_batch)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(net.parameters(),max_norm=5.)
            
            if i % 100 == 0:
                print("train loss", loss.item())
            
            optimizer.step()
        
        total_test_loss = 0.0
        total_examples = 0
        
        all_predictions = []
        correct_predictions = []
        net.eval()
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            with torch.no_grad():
                y_pred = net(X_batch)
                y_pred = y_pred.view(-1) * housing_pipeline.target_std + housing_pipeline.target_mean
                all_predictions.extend(y_pred.tolist())
                correct_predictions.extend(y_batch.tolist())
                loss = test_criterion(y_pred, y_batch)
                total_test_loss += loss.item()
                total_examples += X_batch.size(0)
                
        print(all_predictions[:30])
        print(correct_predictions[:30])
        print("epoch {}".format(epoch + 1))
        print("test loss {}".format(total_test_loss / total_examples))
        print("R^2 ", metrics.r2_score(correct_predictions, all_predictions))
        
        
        
        
    

In [ ]:
device = 'cpu'
housing_net = housing_net.to(device)
train_network(housing_net, train_loader, test_loader, criterion, nn.MSELoss(reduction='sum'), optimizer, n_epoch, device)

In [ ]:
49381818368.0 ** 0.5

In [ ]:
13217517155.224806 ** 0.5

In [ ]:
torch.tensor([1,2,3.]).view(-1,1) - torch.tensor([1,2,3.])

In [ ]:
housing_pipeline.target_std